<a href="https://colab.research.google.com/github/SiriosMEN/Data-science/blob/main/text_tf_idf/text_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим и посмотрим на данные

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostRegressor,CatBoostClassifier,CatBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропуски отсутствуют, все значения заполнены. Целевой признак представлен целым числом, текст - строкой

Проверим распределение классов в целевом признаке

In [ ]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Классы распределены неравномерно, не сбалансированно, необходимо это учитывать при расчете метрики F1.

Напишем функцию для подготовки корпуса текстов к векторизации:

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
def lemmatize(text):
    # Преобразование текста в список слов
    text_list = nltk.word_tokenize(text)
    # Приведение каждого слова в списке к прописному написанию, лемматизация
    lemm_text=[]
    for word in text_list:
        lemm_text.append(lemmatizer.lemmatize(word.lower()))
    # очистка от стоп-слов
    lemm_sw_text = [w for w in lemm_text if w not in stop_words]
    # очистка от посторонних символов
    text_clear = re.sub(r'[^a-zA-Z]',' '," ".join(lemm_sw_text))
    # удаление лишних пробелов после очистки
    text_split = text_clear.split()
    # повторная очистка от стоп слов
    lemm_sw_text2 = [w for w in text_split if w not in stop_words]
    # соединение в единый текст
    return " ".join(lemm_sw_text2)

Создадим новый признак - очищенный и лемматизированный текст:

In [ ]:
data['lemm_clear_text'] = data['text'].apply(lemmatize)

Разделим данные на обучающую и тестовую выборки

In [ ]:
train, test = train_test_split(data,test_size=0.2,random_state=12345)

Выполним векторизацию текстов с помощью TF-IDF, сразу сохраним полученные вектора в обучающие признаки:

In [ ]:
count_tfidf = TfidfVectorizer()
features_train = count_tfidf.fit_transform(train['lemm_clear_text'])
features_test = count_tfidf.transform(test['lemm_clear_text'])

Выделим целевой признак в обучающей и тестовой выборках:

In [ ]:
target_train = train['toxic']
target_test = test['toxic']

### Вывод

Выполнена загрузка данных. Выполнена очистка текста от посторонних символов, приведение текста к нижнему регистру, лемматизация текста. Выполнена векторизация текстов с помощью TF-IDF.

## Обучение

Напишем функцию для обучения моделей и расчета метрики F1:

In [ ]:
def modeling(model):
    model.fit(features_train,target_train)
    prediction_train = model.predict(features_train)
    f1_train = f1_score(target_train,prediction_train)#,average='weighted',labels=np.unique(prediction_train))
    print('Метрика F1 на трейне',f1_train)
    prediction_test = model.predict(features_test)
    f1_test = f1_score(target_test,prediction_test)#,average='weighted',labels=np.unique(prediction_test))
    report = classification_report(target_test,prediction_test)
    print('Метрика F1 на тесте',f1_test)
    print(report)
    return f1_test

Обучим модель логистической регрессии:

In [ ]:
f1_test_LR = modeling(LogisticRegression(solver='liblinear',class_weight='balanced',max_iter=300,random_state=12345))

Метрика F1 на трейне 0.8306323060573858
Метрика F1 на тесте 0.7476836309923458
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     28676
           1       0.66      0.86      0.75      3239

    accuracy                           0.94     31915
   macro avg       0.82      0.90      0.86     31915
weighted avg       0.95      0.94      0.94     31915



Метрика F1 на классе "0" почти 100%, на классе "1" составляет 0,75, что соответствует требованиям задания.

Обучим модель на алгоритме случайный лес:

In [ ]:
f1_test_RFR = modeling(RandomForestClassifier(n_jobs=-1,n_estimators=100,max_depth=12,random_state=12345))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Метрика F1 на трейне 0.0
Метрика F1 на тесте 0.0
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     28676
           1       0.00      0.00      0.00      3239

    accuracy                           0.90     31915
   macro avg       0.45      0.50      0.47     31915
weighted avg       0.81      0.90      0.85     31915



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


У модели "случайный лес" метрика на класс "1" составляет 0, так как метрика Recall составляет 0, при этом метрика Precision составляет 100%, что возможно свидетельствует о переобучении модели.

Обучим модель на алгоритме CatBoost:

In [ ]:
f1_test_CB = modeling(CatBoostClassifier(verbose=10))

Learning rate set to 0.06609
0:	learn: 0.6222009	total: 6.08s	remaining: 1h 41m 11s
10:	learn: 0.3138801	total: 56s	remaining: 1h 23m 52s
20:	learn: 0.2464795	total: 1m 46s	remaining: 1h 22m 38s
30:	learn: 0.2244417	total: 2m 36s	remaining: 1h 21m 39s
40:	learn: 0.2132715	total: 3m 27s	remaining: 1h 20m 42s
50:	learn: 0.2057052	total: 4m 17s	remaining: 1h 19m 57s
60:	learn: 0.1992847	total: 5m 8s	remaining: 1h 19m 13s
70:	learn: 0.1943307	total: 5m 59s	remaining: 1h 18m 27s
80:	learn: 0.1896663	total: 6m 50s	remaining: 1h 17m 41s
90:	learn: 0.1859801	total: 7m 41s	remaining: 1h 16m 51s
100:	learn: 0.1825724	total: 8m 32s	remaining: 1h 16m 1s
110:	learn: 0.1793229	total: 9m 23s	remaining: 1h 15m 14s
120:	learn: 0.1766145	total: 10m 14s	remaining: 1h 14m 24s
130:	learn: 0.1739355	total: 11m 5s	remaining: 1h 13m 37s
140:	learn: 0.1718005	total: 11m 56s	remaining: 1h 12m 47s
150:	learn: 0.1696751	total: 12m 47s	remaining: 1h 11m 55s
160:	learn: 0.1675547	total: 13m 38s	remaining: 1h 11m 7s

Метрика F1 на классе "1" составляет 0,75, что соответствует требованиям задания.

Обучим модель на алгоритме LGBM:

In [ ]:
f1_test_LGBM = modeling(LGBMClassifier(learning_rate=0.25,max_depth=20))

Метрика F1 на трейне 0.8332396072154692
Метрика F1 на тесте 0.7644895815093679
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     28676
           1       0.88      0.67      0.76      3239

    accuracy                           0.96     31915
   macro avg       0.92      0.83      0.87     31915
weighted avg       0.96      0.96      0.96     31915



Метрика F1 на классе "1" составляет 0,76, что даже превышает требования ТЗ.

## Выводы

In [ ]:
results = pd.DataFrame({'Название алгоритма':['Logistic Regression','Random Forest Regressor','CatBoost','Ligth Gradient Boost Machine'],'F1':[f1_test_LR,f1_test_RFR,f1_test_CB,f1_test_LGBM]})
results

,Название алгоритма,F1
0,Logistic Regression,0.747684
1,Random Forest Regressor,0.000000
2,CatBoost,0.748646
3,Ligth Gradient Boost Machine,0.764490


Обучены модели машинного обучения на алгоритмах Логистической регрессии, Случайного леса, CatBoost и LGBM. Наилучшие результаты показала модель LGBM - общая метрика F1 на тесте составила 0,955,  при этом предсказания на классе "1" имеют метрику F1 = 0,76.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны